In [42]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv('/content/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [43]:
df = df.drop(columns = ['PassengerId','Name','Ticket','Cabin'])

x_train,x_test, y_train, y_test = train_test_split(df.drop(columns = ['Survived']), df['Survived'], train_size = 0.8, random_state =0)

x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S
...,...,...,...,...,...,...,...
835,1,female,39.0,1,1,83.1583,C
192,3,female,19.0,1,0,7.8542,S
629,3,male,NaN,0,0,7.7333,Q
559,3,female,36.0,1,0,17.4000,S


In [44]:
tr1 = ColumnTransformer([('Impute_age',SimpleImputer(),[2]), ('Impute_embarked', SimpleImputer(strategy = 'most_frequent'),[6])], remainder = 'passthrough')

tr2 = ColumnTransformer([('encode_sex', OneHotEncoder(sparse_output = False , handle_unknown = 'ignore'),[1,6])], remainder = 'passthrough')

tr3 = ColumnTransformer([('scale', MinMaxScaler(), slice(0,8))])

tr4 = SelectKBest(score_func = chi2, k= 5)

tr5 = DecisionTreeClassifier()

In [45]:
Pipe = Pipeline([('tr1', tr1),
                 ('tr2', tr2),
                 ('tr3', tr3),
                 ('tr4', tr4),
                 ('tr5', tr5)])

In [46]:
Pipe.fit(x_train,y_train)

Pipeline(steps=[('tr1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('Impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tr2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encode_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('tr3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('tr4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x7999fed0d8a0>)),
                ('tr5', DecisionTreeClassifier())])

In [47]:
Pipe.named_steps

{'tr1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Impute_age', SimpleImputer(), [2]),
                                 ('Impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tr2': ColumnTransformer(remainder='passthrough',
                   transformers=[('encode_sex',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'tr3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 8, None))]),
 'tr4': SelectKBest(k=5, score_func=<function chi2 at 0x7999fed0d8a0>),
 'tr5': DecisionTreeClassifier()}

In [48]:
from sklearn import set_config
set_config(display = 'diagram')

from sklearn.metrics import accuracy_score
y_pred = Pipe.predict(x_test)

y_pred

accuracy_score(y_pred,y_test)

0.6759776536312849

In [49]:
import pickle
pickle.dump(Pipe, open('/pipe.pkl','wb'))